In [45]:
from copy import deepcopy


from functools import cache

In [46]:
"""Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0"""

REGS_TEST = {"A": 729, "B": 0, "C": 0}
REGS_TEST_I = (729, 0, 0)
PROG_TEST = [int(i) for i in "0,1,5,4,3,0".split(',')]

"""Register A: 123729
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0"""
REGS_TEST_1 = {"A": 123729, "B": 0, "C": 0}
PROG_TEST_1 = [int(i) for i in "0,3,5,4,3,0".split(',')]
REGS_TEST_I_1 = (123729, 0, 0)

"""Register A: 64012472
Register B: 0
Register C: 0

Program: 2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0"""
REGS = {"A": 64012472, "B": 0, "C": 0}
PROG = [int(i) for i in "2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0".split(',')]
REGS_I = (64012472, 0, 0)

In [47]:
# There are two types of operands; each instruction specifies the type of its operand. 
# The value of a literal operand is the operand itself. 
# For example, the value of the literal operand 7 is the number 7. 
# The value of a combo operand can be found as follows:

# Combo operands 0 through 3 represent literal values 0 through 3.
# Combo operand 4 represents the value of register A.
# Combo operand 5 represents the value of register B.
# Combo operand 6 represents the value of register C.
# Combo operand 7 is reserved and will not appear in valid programs.
def combo(o, regs):
    if 0 <= o <= 3:
        return o
    elif o == 4:
        return regs['A']
    elif o == 5:
        return regs['B']
    elif o == 6:
        return regs['C']
    else:
        raise ValueError("invalid or reserved")
    
def combo_i(o, regs):
    if 0 <= o <= 3:
        return o
    elif o == 4:
        return regs[0]
    elif o == 5:
        return regs[1]
    elif o == 6:
        return regs[2]
    else:
        raise ValueError("invalid or reserved")

# The adv instruction (opcode 0) performs division. The numerator is the value in the A register. 
# The denominator is found by raising 2 to the power of the instruction's combo operand. 
# (So, an operand of 2 would divide A by 4 (2^2); an operand of 5 would divide A by 2^B.) 
# The result of the division operation is truncated to an integer and then written to the A register.
def adv(o, regs, pt, state):
    # num, den = regs['A'], (1 << combo(o, regs))
    # regs['A'] = num // den
    regs['A'] = regs['A'] >> combo(o, regs)
    pt += 2
    return regs, pt, state

def adv_i(o, regs, pt, state):
    # num, den = regs['A'], (1 << combo(o, regs))
    # regs['A'] = num // den
    regs_ = (regs[0] >> combo_i(o, regs), regs[1], regs[2])
    pt += 2
    return regs_, pt, state

# The bxl instruction (opcode 1) calculates the bitwise XOR of register B and the 
# instruction's literal operand, then stores the result in register B.
def bxl(o, regs, pt, state):
    regs['B'] = regs['B'] ^ o
    pt += 2
    return regs, pt, state

def bxl_i(o, regs, pt, state):
    regs_ = (regs[0], regs[1] ^ o, regs[2])
    pt += 2
    return regs_, pt, state

# The bst instruction (opcode 2) calculates the value of its combo operand modulo 8 
# (thereby keeping only its lowest 3 bits), then writes that value to the B register.
def bst(o, regs, pt, state):
    regs['B'] = combo(o, regs) % 8
    pt += 2
    return regs, pt, state

def bst_i(o, regs, pt, state):
    regs_ = (regs[0], combo_i(o, regs) % 8, regs[2])
    pt += 2
    return regs_, pt, state

# The jnz instruction (opcode 3) does nothing if the A register is 0. However, if the A register 
# is not zero, it jumps by setting the instruction pointer to the value of its literal operand; 
# if this instruction jumps, the instruction pointer is not increased by 2 after this instruction.
def jnz(o, regs, pt, state):
    if regs['A'] == 0:
        pt += 2
        return regs, pt, state
    else:
        pt = o
        return regs, pt, state
    
def jnz_i(o, regs, pt, state):
    if regs[0] == 0:
        pt += 2
        return regs, pt, state
    else:
        pt = o
        return regs, pt, state

# The bxc instruction (opcode 4) calculates the bitwise XOR of 
# register B and register C, then stores the result in register B.
# (For legacy reasons, this instruction reads an operand but ignores it.)
def bxc(o, regs, pt, state):
    regs['B'] = regs['B'] ^ regs['C']
    pt += 2
    return regs, pt, state

def bxc_i(o, regs, pt, state):
    regs_ = (regs[0], regs[1] ^ regs[2], regs[2])
    pt += 2
    return regs_, pt, state

# The out instruction (opcode 5) calculates the value of its combo operand modulo 8, 
# then outputs that value. (If a program outputs multiple values, they are separated by commas.)
def out(o, regs, pt, state):
    pt += 2
    state += f"{combo(o, regs) % 8}"
    return regs, pt, state

def out_i(o, regs, pt, state):
    pt += 2
    state += f"{combo_i(o, regs) % 8}"
    return regs, pt, state

# The bdv instruction (opcode 6) works exactly like the adv instruction except
# that the result is stored in the B register. 
# (The numerator is still read from the A register.)
def bdv(o, regs, pt, state):
    # num, den = regs['A'], (1 << combo(o, regs))
    # regs['B'] = num // den
    regs['B'] = regs['A'] >> combo(o, regs)
    pt += 2
    return regs, pt, state

def bdv_i(o, regs, pt, state):
    regs_ = (regs[0], regs[0] >> combo_i(o, regs), regs[2])
    pt += 2
    return regs_, pt, state

# The cdv instruction (opcode 7) works exactly like the adv instruction except 
# that the result is stored in the C register. 
# (The numerator is still read from the A register.)
def cdv(o, regs, pt, state):
    # num, den = regs['A'], (1 << combo(o, regs))
    # regs['C'] = num // den
    regs['C'] = regs['A'] >> combo(o, regs)
    pt += 2
    return regs, pt, state

def cdv_i(o, regs, pt, state):
    regs_ = (regs[0], regs[1], regs[0] >> combo_i(o, regs))
    pt += 2
    return regs_, pt, state

INS = {0: adv, 1: bxl, 2: bst, 3: jnz, 4: bxc, 5: out, 6: bdv, 7: cdv}
INS_I = {0: adv_i, 1: bxl_i, 2: bst_i, 3: jnz_i, 4: bxc_i, 5: out_i, 6: bdv_i, 7: cdv_i}

Part 1

In [48]:
regs = deepcopy(REGS)
prog = PROG
state = ""
pt = 0

while pt < len(prog)-1:
    oc = prog[pt]
    o = prog[pt+1]
    regs, pt, state = INS[oc](o, regs, pt, state)

",".join(list(state))

'1,0,2,0,5,7,2,1,3'

In [49]:
regs = REGS_I
prog = tuple(PROG)
state = ""
pt = 0

while pt < len(prog)-1:
    oc = prog[pt]
    o = prog[pt+1]
    regs, pt, state = INS_I[oc](o, regs, pt, state)

",".join(list(state))

'1,0,2,0,5,7,2,1,3'

Part 2

In [46]:
prog = PROG
orig_state = "".join([str(s) for s in prog])




for i in range(100000):
    regs = deepcopy(REGS)
    regs['A'] = i
    state = ""
    pt = 0
    while pt < len(prog)-1:
        oc = prog[pt]
        o = prog[pt+1]
        regs, pt, state = INS[oc](o, regs, pt, state)
    print(i, oct(i), state)


0 0o0 0
1 0o1 1
2 0o2 2
3 0o3 3
4 0o4 4
5 0o5 4
6 0o6 5
7 0o7 0
8 0o10 01
9 0o11 11
10 0o12 21
11 0o13 31
12 0o14 51
13 0o15 61
14 0o16 11
15 0o17 01
16 0o20 02
17 0o21 12
18 0o22 22
19 0o23 22
20 0o24 62
21 0o25 02
22 0o26 52
23 0o27 02
24 0o30 03
25 0o31 13
26 0o32 23
27 0o33 23
28 0o34 73
29 0o35 23
30 0o36 13
31 0o37 03
32 0o40 04
33 0o41 14
34 0o42 34
35 0o43 14
36 0o44 04
37 0o45 44
38 0o46 54
39 0o47 04
40 0o50 04
41 0o51 14
42 0o52 34
43 0o53 14
44 0o54 14
45 0o55 64
46 0o56 14
47 0o57 04
48 0o60 05
49 0o61 15
50 0o62 35
51 0o63 05
52 0o64 25
53 0o65 05
54 0o66 55
55 0o67 05
56 0o70 00
57 0o71 10
58 0o72 30
59 0o73 00
60 0o74 30
61 0o75 20
62 0o76 10
63 0o77 00
64 0o100 001
65 0o101 001
66 0o102 001
67 0o103 701
68 0o104 401
69 0o105 401
70 0o106 501
71 0o107 001
72 0o110 011
73 0o111 011
74 0o112 011
75 0o113 711
76 0o114 511
77 0o115 611
78 0o116 111
79 0o117 011
80 0o120 021
81 0o121 021
82 0o122 021
83 0o123 621
84 0o124 621
85 0o125 021
86 0o126 521
87 0o127 021
88 0o130 0

In [ ]:
def prints_i(i):
    for j in range(7):
        regs = (j, 0, 0)
        state = ""
        pt = 0
        while pt < len(prog)-1:
            oc = prog[pt]
            o = prog[pt+1]
            regs, pt, state = INS_I[oc](o, regs, pt, state)
        if state == str(i):
            return j
    return -1



In [61]:
[1, 2, 3, 4][-4:]

[1, 2, 3, 4]

In [82]:
# 2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0

i = 1
rA = ""
orig_state = "".join([str(s) for s in prog])
while i <= 2:
    st_end = orig_state[-i:]
    print("ST END", st_end)
    for j in range(8):
        rA_tmp = rA + str(j)
        print(rA_tmp)

        rA_tmp_int = int(rA_tmp + (16-i)*'0', 8)
        regs = (rA_tmp_int, 0, 0)
        state = ""
        pt = 0
        while pt < len(prog)-1:
            oc = prog[pt]
            o = prog[pt+1]
            regs, pt, state = INS_I[oc](o, regs, pt, state)
        # if i == 2: 
        #     print(rA_tmp_int, state)
        print("state: ", state)
        if state[-i:] == st_end:
            # print("BLA")
            rA = rA_tmp
            break
    i += 1

ST END 0
0
state:  0
BLA
ST END 30
00
state:  0
01
state:  000000000004001
02
state:  000000000000102
03
state:  000000000004103
04
state:  000000000000204
05
state:  000000000004204
06
state:  000000000000305
07
state:  000000000004300


In [87]:
for i in range(8):
    for j in range(8):
        # rA = (str(i) + str(j)) + 14*'0'
        rA = (str(i) + str(j))
        regs = (int(rA, 8), 0, 0)
        state = ""
        pt = 0
        while pt < len(prog)-1:
            oc = prog[pt]
            o = prog[pt+1]
            regs, pt, state = INS_I[oc](o, regs, pt, state)
        print(rA, state)


00 0
01 1
02 2
03 3
04 4
05 4
06 5
07 0
10 01
11 11
12 21
13 31
14 51
15 61
16 11
17 01
20 02
21 12
22 22
23 22
24 62
25 02
26 52
27 02
30 03
31 13
32 23
33 23
34 73
35 23
36 13
37 03
40 04
41 14
42 34
43 14
44 04
45 44
46 54
47 04
50 04
51 14
52 34
53 14
54 14
55 64
56 14
57 04
60 05
61 15
62 35
63 05
64 25
65 05
66 55
67 05
70 00
71 10
72 30
73 00
74 30
75 20
76 10
77 00


In [133]:
def run(prog: tuple, rA: int, built_str: str):
    regs = (rA, 0, 0)
    state = ""
    pt = 0
    while pt < len(prog)-1:
        oc = prog[pt]
        o = prog[pt+1]
        regs, pt, state = INS_I[oc](o, regs, pt, state)
        if state and not built_str.startswith(state):
            return False
    return state == built_str

prog = tuple(PROG)
built_str = ""
rA_str = "30"
rA = int(rA_str, 8)
run(prog, rA, built_str)

True

In [134]:
prog = PROG
orig_state = "".join([str(s) for s in prog])


In [173]:
@cache
def compute(prog, potential_strings, orig_str):
    
    print(potential_strings)
    rA_str, built_str = potential_strings[-1]

    if run(prog, int(rA_str, 8), built_str) and built_str == orig_str:
        print(f'{rA_str}')
        raise RuntimeError("End")
    
    if len(built_str) == len(orig_str):
        potential_strings = potential_strings[:-1]
        compute(prog, potential_strings, orig_str)

    can_extend = False
    for i in range(8):
        
        new_rA_str = str(i) + rA_str
        rA = int(new_rA_str, 8)
        new_built_str = built_str + orig_str[len(built_str)]

        # print(f"{i}{rA_str} - {new_built_str}")

        if run(prog, rA, new_built_str):
            t = (new_rA_str, new_built_str)
            potential_strings += (t,)
            compute(prog, potential_strings, orig_str)
            can_extend = True
        else:
            continue
    
    if not can_extend:
        potential_strings = potential_strings[:-1]
        compute(prog, potential_strings, orig_str)

In [176]:
prog = tuple(PROG)

orig_str = "551"
# orig_str = "".join([str(s) for s in prog])
potential_strings_init = tuple((str(i), orig_str[0]) for i in range(7, -1, -1))

compute(prog, potential_strings_init, orig_str)

(('7', '5'), ('6', '5'), ('5', '5'), ('4', '5'), ('3', '5'), ('2', '5'), ('1', '5'), ('0', '5'))
(('7', '5'), ('6', '5'), ('5', '5'), ('4', '5'), ('3', '5'), ('2', '5'), ('1', '5'))
(('7', '5'), ('6', '5'), ('5', '5'), ('4', '5'), ('3', '5'), ('2', '5'))
(('7', '5'), ('6', '5'), ('5', '5'), ('4', '5'), ('3', '5'))
(('7', '5'), ('6', '5'), ('5', '5'), ('4', '5'))
(('7', '5'), ('6', '5'), ('5', '5'))
(('7', '5'), ('6', '5'))
(('7', '5'), ('6', '5'), ('66', '55'))
(('7', '5'), ('6', '5'))
(('7', '5'), ('6', '5'), ('66', '55'))
(('7', '5'), ('6', '5'))
(('7', '5'), ('6', '5'), ('66', '55'))
(('7', '5'), ('6', '5'))
(('7', '5'), ('6', '5'), ('66', '55'))
(('7', '5'), ('6', '5'))
(('7', '5'), ('6', '5'), ('66', '55'))
(('7', '5'), ('6', '5'))
(('7', '5'), ('6', '5'), ('66', '55'))
(('7', '5'), ('6', '5'))
(('7', '5'), ('6', '5'), ('66', '55'))
(('7', '5'), ('6', '5'))
(('7', '5'), ('6', '5'), ('66', '55'))
(('7', '5'), ('6', '5'))
(('7', '5'), ('6', '5'), ('66', '55'))
(('7', '5'), ('6', '5'

RecursionError: maximum recursion depth exceeded while getting the repr of an object